# 11.3 使用猴子补丁在运行时实现协议

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'R B W Y'.split()

    def __init__(self) -> None:
        self._cards = [Card(rank, suit) for suit in self.suits 
                                        for rank in self.ranks]

    # 调用len(self)
    def __len__(self):
        return len(self._cards)

    # 调用self[position]
    def __getitem__(self, position):
        return self._cards[position]


上面的 FrenchDeck 类有一个重大缺陷：无法洗牌。

In [2]:
from random import shuffle
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

'FrenchDeck' object does not support item assignment，对象不支持为元素赋值。这个问题的原因是 FrenchDeck 只实现了 _不可变_ 的序列协议。可变的序列还必须提供 \_\_setitem__ 方法。

Python 是动态语言，因此我们可以在运行时修正这个问题，甚至还可以在交互式控制台中。

In [3]:
# 为 FrenchDeck 打猴子补丁，把它变成可变的，让 random.shuffle 可以使用
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='A', suit='W'),
 Card(rank='J', suit='B'),
 Card(rank='6', suit='R'),
 Card(rank='5', suit='Y'),
 Card(rank='3', suit='R')]

# 11.4 鸭子类型和抽象基类

In [ ]:
# 使用鸭子类型处理单个字符串或由字符串组成的可迭代对象
try:    # 假设是单个字符串
    field_names = field_names.replace(',', ' ').split() # 把 , 替换成空格然后拆分成列表
except AttributeError:  # 如果它不是字符串则引发一场
    pass    # 假设是由名称组成的可迭代对象
field_names = tuple(field_names)    # 为了确保是可迭代对象，也为了保存一份副本，使用所得值创建一个元组

# 11.5 定义抽象基类的子类

In [ ]:
# FrenchDeck2, collections.MutableSequence 的子类
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'R B W Y'.split()

    def __init__(self) -> None:
        self._cards = [Card(rank, suit) for suit in self.suits 
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    # 为了支持洗牌，只需实现 __setitem__ 方法
    def __setitem__(self, position, value):
        self._cards[position] = value
    
    # 继承 MutableSequence 的类必须实现 __delitem__ 方法，这是 MutableSequence 的一个抽象方法
    def __delitem__(self, position):
        self._cards[position]
    
    # 此外，要实现 insetr 方法，这是 MutableSequence 的第三个抽象方法
    def insert(self, positon, value):
        self._cards.insert(positon, value)

导入时(加载并编译 .py 模块时)，Python 不会加尼察抽象方法的实现，在运行时实例化的时候猜会真正检查。

# 11.6 标准库中的抽象基类
大多数抽象基类都在 collections.abc 模块中，不过其他地方也有。比如 numbers 和 io 包中有一些抽象基类

## 11.6.1 collections.abc 中的抽象基类  
  
<img src=.\Figure11-3.png width=600 height=300>


* Iterable、Contatiner 和 sized

各个集合都应该继承这三个抽象基类，或者至少实现兼容的协议。Iterable 通过 \_\_iter__ 方法支持迭代，Contatiner 通过 \_\_contains__ 方法支持 in 运算符，Sized 通过 \_\_len__ 方法支持 len() 函数。

* Sequence、Mapping 和 Set

这三个是主要的不可变集合类型，而且各有各的可变子类。

* MappingView

在 Python 3 中，映射方法 .items()、.keys() 和 .values() 返回的对象分别是 ItemsView、KeysView 和 ValueView 的实例，前两个类还从 Set 类继承了丰富的接口。

* Callable 和 Hashable

这两个抽象记基类与集合没有太大关系，只不过因为 collections.abc 是标准库中定义抽象基类的第一个模块，它们又太重要了，因此才把他们放入 collections.abc 模块中  
这两个抽象函数的主要作用是为内组织函数 isinstance 提供支持，以一种安全的方式判断对象能不能调用或散列

* Iterator

注意它是 Iterable 的子类
    

## 11.6.2 抽象基类的数字塔

numbers 包定义的是 "数字塔" （各个抽象基类的层次结构是线性的），其中 Number 是位于最顶端的超类，随后是 Complex 子类，依次往下，最低端的是 Integral 类：
* Number
* Complex
* Real
* Rational
* Integral

因此，如果想检查一个数是不是整数，可以使用 isinstance(x, numbers.Integral)，这样的代码能接受 int，bool(int 的子类)，或者外部库使用 numbers 抽象基类注册的其他类型。  
与之类似，如果一个值可能是浮点数类型，可以使用 isinstance(x, numbers.Real)检查，这样代码能接受 int、bool、float、fractions.Fraction，或者外部库提供的非复数类型。  

# 11.7 定义并使用一个抽象基类

有这样一个场景，需要在网站或移动应用中显示随机广告，但在整个广告清单轮转一边之前，不重复显示广告。假设我们在构建一个广告管理框架，名为 ADAM。它的职责之一是，支持用户提供随机挑选的无重复类。为了让用户明确理解"随机挑选的无重复"组件是什么意思，我们将定义一个抽象基类

收到“栈”和“队列”启发，我将使用现实世界中的物品命名这个抽象基类：宾果机和彩票机是随即从有限的集合中挑选物品的机器，选出的物品没有重复，直到选完为止。

我们把这个抽象基类命名为 Tombola，这是打乱数字的滚动容器的意大利名。它有四个方法，其中两个是抽象方法
* .load(...)：把元素放入容器
* .pick()：从容器中随机拿出一个元素，返回选中的元素

另外两个是具体方法
* .loaded()：如果容器中至少有一个元素，返回 True
* .inspect()：返回一个有序元组，由容器中的现有元素构成，不会修改容器的内容(内部顺序不保留)

In [4]:
import abc

class Tombola(abc.ABC): # 自己定义的抽象基类要继承 abc.ABC

    @abc.abstractmethod # 抽象方法用 @abc.abstractmethod 标记，并且定义体中通常只有文档字符串
    def load(self, iterable):
        """从可迭代对象中添加元素"""
    
    @abc.abstractmethod
    def pick(self):
        """随机删除元素，然后将其返回。
        
        如果实例为空，这个方法应抛出 LookupError
        """

    def loaded(self):   # 抽象基类可以包含具体方法
        """如果至少有一个元素，返回 True，否则返回 False"""
        return bool(self.inspect()) # 抽象基类的具体方法只能依赖抽象基类定义的接口
    
    def inspect(self): 
        """返回一个有序元组，由当前元素构成"""
        items = []
        while True: # 我们不知道子类如何储存元素，不过为了得到结果，我们可以不断调用 pick() 方法，把 Tombola 清空
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)    # 然后再用 load() 把元素放回去
        return tuple(sorted(items))

抽象方法可以有实现代码，即时实现了，子类也必须覆盖抽象方法，但是在子类中可以使用 super() 函数调用抽象方法，为它添加功能而不是从头开始实现。

异常类的部分层次结构
```
BaseException
 ├── SystemExit
 ├── KeyboardInterrupt
 ├── GeneratorExit
 └── Exception
      ├── StopIteration
      ├── ArithmeticError
      │    ├── FloatingPointError
      │    ├── OverflowError
      │    └── ZeroDivisionError
      ├── AssertionError
      ├── AttributeError
      ├── BufferError
      ├── EOFError
      ├── ImportError
      ├── LookupError
      │    ├── IndexError
      │    └── KeyError
      ├── MemoryError
      ... etc.
```


下面创建一个有缺陷的实现来糊弄 Tombola

In [5]:
class Fake(Tombola):
    def pick(self):
        return 13

Fake    # 可以创建类

__main__.Fake

In [6]:
f = Fake()  #实例化时抛出错误

TypeError: Can't instantiate abstract class Fake with abstract method load

## 11.7.1 抽象基类语法详解

除了 @abstractmechod 外，abc 模块还定义了 @abstractclassmethod、@abstractstaticmethod 和 @abstractpropety 三个装饰器，不过它们从 Python 3.3 起废弃了，因为装饰器可以嵌套使用  
比如声明抽象类方法的推荐方法

In [ ]:
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, args):
        pass

## 11.7.2 定义 Tombola 抽象基类的子类

In [7]:
# BingoCage 类
import random

class BingoCage(Tombola):   # 指明 BingoCage 类扩展 Tombola 类

    def __init__(self, items) -> None:
        self._randomizer = random.SystemRandom()    # random.SystemRandom 使用 os.urandom(...) 函数实现 random API。
        self._items = []
        self.load(items)    # 委托 load 方法实现初始加载
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)   # 没有使用 random.shuffle() 而是使用 SystemRandom 实例的 .shuffle() 方法
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [8]:
bingo = BingoCage([1,2,3])

In [ ]:
# LotteryBlower，Tombola 的另一个子类
import random

class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)    # 把参数构建成列表

    def load(self, iterable):
        self._balls.extend(iterable)
    
    def pick(self):
        try:
            position = random.randrange(len(self._balls))   
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)
    
    def loaded(self):   # 覆盖 loaded 方法，避免使用 inspect 方法
        return bool(self._balls)
    
    def inspect(self):  # 覆盖 inspect 方法
        return tuple(sorted(self._balls))

## 11.7.3 Tombola 的虚拟子类

白鹅类型的一个基本特性：即便不继承，也有办法把一个类注册为抽象基类的虚拟子类。这样做时，我们保证注册的类忠实地实现了抽象基类定义的接口，而 Python 会相信我们，从而不做检查。

注册虚拟子类的方式是在抽象基类上调用 register 方法，这么做后，注册的类会变成抽象基类的虚拟子类。而且 issubclass 和 isinstance 等函数都能识别，但是注册的类不会从抽象基类中继承任何方法和属性

register 方法通常作为普通的函数调用，不过也可以作为装饰器使用

In [11]:
# TomboList 是 Tombola 的虚拟子类
from random import randrange

@Tombola.register   # 把 TomboList 注册为 Tombola 的虚拟子类
class TomboList(list):  # TomboList 扩展 List
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend

    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(sorted(self))

# Tombola.register(TomboList)

类的继承关系在一个特殊的类属性中指定 —— \_\_mro__，即方法解析顺序(Method Resolution Order)。这个属性的作用很简单，按顺序列出类及其超类，Python 会按照这个顺序搜索方法。  
查看 TomboList 的 \_\_mro__ 属性，会发现它只列出了最真实的超类，即 list 和 object

In [13]:
TomboList.__mro__

(__main__.TomboList, list, object)

# 11.8 Tombola 子类的测试方法

用它们内省类的继承关系
* \_\_subclassess__()   ：返回类的直接子类列表，不含虚拟子类，这里的直接是指内存中存在的直接子代，只要导入切是子代就会包含在返回中
* _abc_registry ：只有抽象基类有这个数据属性，其值是一个 WeakSet 对象，即抽象类注册的虚拟子类的弱引用

# 11.10 鹅的行为有可能像鸭子